In [18]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=30

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
f=generate_random_poly(v)


# unit sphere constraint
m=1
q=floor(Int64,n/m)
R=ones(Float64,m)./m
T=[(j-1)*q+1:j*q for j in 1:m-1]
append!(T,[(m-1)*q+1:n])

g=[R[j]-sum(x[T[j]].^2) for j in 1:m]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/4)

h=Vector{Polynomial{true,Float64}}(undef,l)
randx=[2*rand(length(T[j])).-1 for j in 1:m]# create a feasible solution
randx=[sqrt(R[j])*rand(1)[1]*randx[j]/norm(randx[j]) for j in 1:m]
randx=vcat(randx...)

for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=30
Number of inequality constraints: m=1
Number of equality constraints: l=8


In [19]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=false);

In [20]:
k=2
println("Relaxed order: k=",k)

Relaxed order: k=2


In [21]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                         maxit=Int64(1e6),tol=1e-3,
                         use_eqcons_to_get_constant_trace=false,
                         check_tol_each_iter=true)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=

2
  Size of the largest block: s^max=496
  Number of equality trace constraints: zeta=81345
Modeling time:
 13.547388 seconds (40.77 M allocations: 12.268 GiB, 19.60% gc time)
iter=1   val=-7.900665517995127   gap=0.09348850571042829   feas=2.0641031077310745
iter=2   val=-5.030201088368732   gap=0.32486988906486797   feas=0.9213830583765027
iter=4   val=-4.728668801332295   gap=0.37856002838635305   feas=0.6178990771270882
iter=8   val=-4.473712192854633   gap=0.3293872218061929   feas=0.5287384269665354
iter=16   val=-4.193311298579527   gap=0.15414313139036356   feas=0.2874524779823411
iter=32   val=-4.1168190391458355   gap=0.0012986328879876574   feas=0.08595089324034491
iter=64   val=-4.19288027363604   gap=0.008385903157248551   feas=0.020922067037319685
iter=128   val=-4.221619067161086   gap=0.0022227774282339396   feas=0.005256420028383287
iter=256   val=-4.229167019074469   gap=0.0005120193235030469   feas=0.001283716544546903
iter=295   val=-4.229750711237803   gap=0.000619

-4.229750711237803

In [22]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val2=ctpPOP.POP_dense_SOS(x,f,g,h,k,tol=1e-3,solver="COSMO")

**SumOfSquares+Mosek:


------------------------------------------------------------------
          COSMO v0.7.7 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2020
------------------------------------------------------------------

Problem:  x ∈ R^{127721},
          constraints: A ∈ R^{170128x127721} (2230513 nnz),
          matrix size to factor: 297849x297849,
          Floating-point precision: Float64
Sets:     PsdConeTriangle of dim: 123256
          ZeroSet of dim: 46376
          PsdConeTriangle of dim: 496
Settings: ϵ_abs = 1.0e-04, ϵ_rel = 1.0e-04,
          ϵ_prim_inf = 1.0e-06, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 2500,
          scaling iter = 10 (on),
          check termination every 40 iter,
          check infeasibility every 40 iter,
          KKT system solver: COSMO.QdldlKKTSolver
Setup Time: 21463.82ms

Iter:	Objective:	Primal Res:	Dual Res:	Rho:
1	-1.7685e+01	1.6115e

-4.2270929401842725

In [15]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val2=ctpPOP.POP_dense_SOS(x,f,g,h,k,tol=1e-3)

**SumOfSquares+Mosek:


Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 10626           
  Cones                  : 0               
  Scalar variables       : 1156            
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.03    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

-3.6226791519503188

In [23]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val=ctpPOP.POP_dense_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                         tol=1e-3,use_eqcons_to_get_constant_trace=false)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=

2
  Size of the largest block: s^max=496
  Number of equality trace constraints: zeta=81345
Modeling time:
 12.076548 seconds (40.77 M allocations: 12.268 GiB, 18.96% gc time)
---------------
| Parameters: |
---------------
n:       81345
maxtime: 300000.000000
na:      2
mcu:     5
mc:      7
rpar: 
ipar: 
 Entry to LMBM:
 NIT=    1  NFE=    1  F= 0.30493505D+00  WK= 0.2299D+00  QK= 0.1149D+00
 NIT=    2  NFE=    2  F= 0.26010729D+00  WK= 0.9699D-01  QK= 0.7931D-01
 NIT=    3  NFE=    3  F= 0.24689012D+00  WK= 0.3451D-01  QK= 0.3197D-01
 NIT=    4  NFE=    4  F= 0.21436501D+00  WK= 0.3702D+00  QK= 0.2731D-01
 NIT=    5  NFE=    5  F= 0.21436501D+00  WK= 0.5461D-01  QK= 0.2731D-01
 NIT=    6  NFE=    6  F= 0.21436501D+00  WK= 0.4581D-01  QK= 0.2333D-01
 NIT=    7  NFE=    7  F= 0.20302516D+00  WK= 0.1675D-01  QK= 0.8757D-01
 NIT=    8  NFE=    8  F= 0.20302516D+00  WK= 0.9811D-02  QK= 0.5842D-01
 NIT=    9  NFE=   10  F= 0.20256915D+00  WK= 0.1404D-01  QK= 0.1246D+00
 NIT=   10  NFE=  

-4.2363647917523055

In [24]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     7440
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:     4215

Total number of variables............................:       30
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

-4.231795351271079